In [38]:
import numpy as np
import time

'''
example state: np.array([['x','x',' '],['o','o',' '],[' ',' ',' ']])
example action: {'player': 'x', 'cell': (0,2)}
'''

def state2board(state, board=" {0} | {1} | {2} \n-----------\n {3} | {4} | {5} \n-----------\n {6} | {7} | {8} "):
    return board.format(*state.flatten())

def state_transition2newstate(state,transition):
    newstate = state.copy()
    newstate[transition['action']] = transition['player']
    return newstate

def state2player(state):
    if (state=='x').sum() > (state=='o').sum():
        return 'o'
    else:
        return 'x'
    
def state2iswin(state):
    x = (state == 'x')
    o = (state == 'o')
    
    x_triples = x.prod(axis=0).sum() + x.prod(axis=1).sum() + np.diag(x).prod() + np.diag(x.transpose()).prod()
    o_triples = o.prod(axis=0).sum() + o.prod(axis=1).sum() + np.diag(o).prod() + np.diag(o.transpose()).prod()
    
    if x_triples + o_triples > 0:
        return True
    else:
        return False
    
def state2actions(state):
    if state2iswin(state):
        return []
    else:
        return zip(np.where(state == ' ')[0], np.where(state == ' ')[1])

def state2transitions(state):
    return [{'player': state2player(state), 'action': action} for action in state2actions(state)]

def state2hash(state):
    return ''.join(state.flatten())

def hash2state(h):
    return np.array(list(h)).reshape(3,3)

def hash2transitions(h):
    return state2transitions(hash2state(h))

def hash_transition2newhash(h,transition):
    return state2hash(state_transition2newstate(hash2state(h),transition))
    
def level2nextlevel(level):
    nextlevel = []
    linkages = []
    for ih,h in enumerate(level):
        for itransition,transition in enumerate(hash2transitions(h)):
            newhash = hash_transition2newhash(h,transition)
            if newhash in nextlevel:
                linkages.append((ih,nextlevel.index(newhash)))
            else:
                nextlevel.append(newhash)
                linkages.append((ih,itransition))
                
    return nextlevel, linkages

def hash2player(h):
    return state2player(hash2state(h))

def hash2iswin(h):
    return state2iswin(hash2state(h))
#def main():
    

'''
def unlist(l):
    return [item for sublist in l for item in sublist]
'''

'\ndef unlist(l):\n    return [item for sublist in l for item in sublist]\n'

In [45]:
print "BUILDING TREE..."
levels = {0: [state2hash(np.array([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']]))]}
linkages = {}
for i in range(10):
    start_time = time.clock()
    [levels[i+1], linkages[i]] = level2nextlevel(levels[i])
    end_time = time.clock()
    print '   level {0} constructed | duration: {1} | num_states: {2}'.format(*[i+1,end_time-start_time,len(levels[i+1])])
levels.pop(10)

print "\nINITIALIZING VALUES..."
values_x = {}
values_o = {}
for i in range(10):
    values_x[i] = []
    values_o[i] = []
    for h in levels[i]:
        if hash2iswin(h):
            if hash2player(h) == 'x':
                values_x[i].append(1)
                values_o[i].append(0)
            elif hash2player(h) == 'o':
                values_o[i].append(1)
                values_x[i].append(0)
        elif i==9:
            values_x[i].append(0)
            values_o[i].append(0)
        else:
            values_x[i].append(np.random.rand())
            values_o[i].append(np.random.rand())
    
tree = {}
for i in range(10):
    tree[i] = {i:{'hashes': levels[i], 'transitions': linkages[i], 'values_x': values_x[i], 'values_o': values_o[i]}}

BUILDING TREE...
   level 1 constructed | duration: 0.000783999999999 | num_states: 9
   level 2 constructed | duration: 0.006233 | num_states: 72
   level 3 constructed | duration: 0.049526 | num_states: 252
   level 4 constructed | duration: 0.068659 | num_states: 756
   level 5 constructed | duration: 0.233115 | num_states: 1260
   level 6 constructed | duration: 0.400207 | num_states: 1540
   level 7 constructed | duration: 0.348983 | num_states: 1155
   level 8 constructed | duration: 0.130711 | num_states: 420
   level 9 constructed | duration: 0.085012 | num_states: 84
   level 10 constructed | duration: 0.012459 | num_states: 0
INITIALIZING VALUES...


In [42]:
tree[1]

{1: {'hashes': ['x        ',
   ' x       ',
   '  x      ',
   '   x     ',
   '    x    ',
   '     x   ',
   '      x  ',
   '       x ',
   '        x'],
  'transitions': [(0, 0),
   (0, 1),
   (0, 2),
   (0, 3),
   (0, 4),
   (0, 5),
   (0, 6),
   (0, 7),
   (1, 0),
   (1, 1),
   (1, 2),
   (1, 3),
   (1, 4),
   (1, 5),
   (1, 6),
   (1, 7),
   (2, 0),
   (2, 1),
   (2, 2),
   (2, 3),
   (2, 4),
   (2, 5),
   (2, 6),
   (2, 7),
   (3, 0),
   (3, 1),
   (3, 2),
   (3, 3),
   (3, 4),
   (3, 5),
   (3, 6),
   (3, 7),
   (4, 0),
   (4, 1),
   (4, 2),
   (4, 3),
   (4, 4),
   (4, 5),
   (4, 6),
   (4, 7),
   (5, 0),
   (5, 1),
   (5, 2),
   (5, 3),
   (5, 4),
   (5, 5),
   (5, 6),
   (5, 7),
   (6, 0),
   (6, 1),
   (6, 2),
   (6, 3),
   (6, 4),
   (6, 5),
   (6, 6),
   (6, 7),
   (7, 0),
   (7, 1),
   (7, 2),
   (7, 3),
   (7, 4),
   (7, 5),
   (7, 6),
   (7, 7),
   (8, 0),
   (8, 1),
   (8, 2),
   (8, 3),
   (8, 4),
   (8, 5),
   (8, 6),
   (8, 7)],
  'values_o': 0.6798305664960202,


In [44]:
import random
from copy import copy, deepcopy
import csv
import matplotlib.pyplot as plt

EMPTY = 0
PLAYER_X = 1
PLAYER_O = 2
DRAW = 3

def emptystate():
    return [[EMPTY,EMPTY,EMPTY],[EMPTY,EMPTY,EMPTY],[EMPTY,EMPTY,EMPTY]]

def gameover(state):
    for i in range(3):
        if state[i][0] != EMPTY and state[i][0] == state[i][1] and state[i][0] == state[i][2]:
            return state[i][0]
        if state[0][i] != EMPTY and state[0][i] == state[1][i] and state[0][i] == state[2][i]:
            return state[0][i]
    if state[0][0] != EMPTY and state[0][0] == state[1][1] and state[0][0] == state[2][2]:
        return state[0][0]
    if state[0][2] != EMPTY and state[0][2] == state[1][1] and state[0][2] == state[2][0]:
        return state[0][2]
    for i in range(3):
        for j in range(3):
            if state[i][j] == EMPTY:
                return EMPTY
    return DRAW

def last_to_act(state):
    countx = 0
    counto = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] == PLAYER_X:
                countx += 1
            elif state[i][j] == PLAYER_O:
                counto += 1
    if countx == counto:
        return PLAYER_O
    if countx == (counto + 1):
        return PLAYER_X
    return -1


def enumstates(state, idx, agent):
    if idx > 8:
        player = last_to_act(state)
        if player == agent.player:
            agent.add(state)
    else:
        winner = gameover(state)
        if winner != EMPTY:
            return
        i = idx / 3
        j = idx % 3
        for val in range(3):
            state[i][j] = val
            enumstates(state, idx+1, agent)

class Agent(object):
    def __init__(self, player, verbose = False, lossval = 0, learning = True):
        self.values = {}
        self.player = player
        self.verbose = verbose
        self.lossval = lossval
        self.learning = learning
        self.epsilon = 0.1
        self.alpha = 0.99
        self.prevstate = None
        self.prevscore = 0
        self.count = 0
        enumstates(emptystate(), 0, self)

    def episode_over(self, winner):
        self.backup(self.winnerval(winner))
        self.prevstate = None
        self.prevscore = 0

    def action(self, state):
        r = random.random()
        if r < self.epsilon:
            move = self.random(state)
            self.log('>>>>>>> Exploratory action: ' + str(move))
        else:
            move = self.greedy(state)
            self.log('>>>>>>> Best action: ' + str(move))
        state[move[0]][move[1]] = self.player
        self.prevstate = self.statetuple(state)
        self.prevscore = self.lookup(state)
        state[move[0]][move[1]] = EMPTY
        return move

    def random(self, state):
        available = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == EMPTY:
                    available.append((i,j))
        return random.choice(available)

    def greedy(self, state):
        maxval = -50000
        maxmove = None
        if self.verbose:
            cells = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == EMPTY:
                    state[i][j] = self.player
                    val = self.lookup(state)
                    state[i][j] = EMPTY
                    if val > maxval:
                        maxval = val
                        maxmove = (i, j)
                    if self.verbose:
                        cells.append('{0:.3f}'.format(val).center(6))
                elif self.verbose:
                    cells.append(NAMES[state[i][j]].center(6))
        if self.verbose:
            print BOARD_FORMAT.format(*cells)
        self.backup(maxval)
        return maxmove

    def backup(self, nextval):
        if self.prevstate != None and self.learning:
            self.values[self.prevstate] += self.alpha * (nextval - self.prevscore)

    def lookup(self, state):
        key = self.statetuple(state)
        if not key in self.values:
            self.add(key)
        return self.values[key]

    def add(self, state):
        winner = gameover(state)
        tup = self.statetuple(state)
        self.values[tup] = self.winnerval(winner)

    def winnerval(self, winner):
        if winner == self.player:
            return 1
        elif winner == EMPTY:
            return 0.5
        elif winner == DRAW:
            return 0
        else:
            return self.lossval

    def printvalues(self):
        vals = deepcopy(self.values)
        for key in vals:
            state = [list(key[0]),list(key[1]),list(key[2])]
            cells = []
            for i in range(3):
                for j in range(3):
                    if state[i][j] == EMPTY:
                        state[i][j] = self.player
                        cells.append(str(self.lookup(state)).center(3))
                        state[i][j] = EMPTY
                    else:
                        cells.append(NAMES[state[i][j]].center(3))
            print BOARD_FORMAT.format(*cells)

    def statetuple(self, state):
        return (tuple(state[0]),tuple(state[1]),tuple(state[2]))

    def log(self, s):
        if self.verbose:
            print s

class Human(object):
    def __init__(self, player):
        self.player = player

    def action(self, state):
        printboard(state)
        action = raw_input('Your move? ')
        return (int(action.split(',')[0]),int(action.split(',')[1]))

    def episode_over(self, winner):
        if winner == DRAW:
            print 'Game over! It was a draw.'
        else:
            print 'Game over! Winner: Player {0}'.format(winner)

def play(agent1, agent2):
    state = emptystate()
    for i in range(9):
        if i % 2 == 0:
            move = agent1.action(state)
        else:
            move = agent2.action(state)
        state[move[0]][move[1]] = (i % 2) + 1
        winner = gameover(state)
        if winner != EMPTY:
            return winner
    return winner

def measure_performance_vs_random(agent1, agent2):
    epsilon1 = agent1.epsilon
    epsilon2 = agent2.epsilon
    agent1.epsilon = 0
    agent2.epsilon = 0
    agent1.learning = False
    agent2.learning = False
    r1 = Agent(1)
    r2 = Agent(2)
    r1.epsilon = 1
    r2.epsilon = 1
    probs = [0,0,0,0,0,0]
    games = 100
    for i in range(games):
        winner = play(agent1, r2)
        if winner == PLAYER_X:
            probs[0] += 1.0 / games
        elif winner == PLAYER_O:
            probs[1] += 1.0 / games
        else:
            probs[2] += 1.0 / games
    for i in range(games):
        winner = play(r1, agent2)
        if winner == PLAYER_O:
            probs[3] += 1.0 / games
        elif winner == PLAYER_X:
            probs[4] += 1.0 / games
        else:
            probs[5] += 1.0 / games
    agent1.epsilon = epsilon1
    agent2.epsilon = epsilon2
    agent1.learning = True
    agent2.learning = True
    return probs

def measure_performance_vs_each_other(agent1, agent2):
    #epsilon1 = agent1.epsilon
    #epsilon2 = agent2.epsilon
    #agent1.epsilon = 0
    #agent2.epsilon = 0
    #agent1.learning = False
    #agent2.learning = False
    probs = [0,0,0]
    games = 100
    for i in range(games):
        winner = play(agent1, agent2)
        if winner == PLAYER_X:
            probs[0] += 1.0 / games
        elif winner == PLAYER_O:
            probs[1] += 1.0 / games
        else:
            probs[2] += 1.0 / games
    #agent1.epsilon = epsilon1
    #agent2.epsilon = epsilon2
    #agent1.learning = True
    #agent2.learning = True
    return probs

'''
if __name__ == "__main__":
    p1 = Agent(1, lossval = -1)
    p2 = Agent(2, lossval = -1)
    r1 = Agent(1, learning = False)
    r2 = Agent(2, learning = False)
    r1.epsilon = 1
    r2.epsilon = 1
    series = ['P1-Win','P1-Lose','P1-Draw','P2-Win','P2-Lose','P2-Draw']
    #series = ['P1-Win', 'P2-Win', 'Draw']
    colors = ['r','b','g','c','m','b']
    markers = ['+', '.', 'o', '*', '^', 's']
    f = open('results.csv', 'wb')
    writer = csv.writer(f)    
    writer.writerow(series)
    perf = [[] for _ in range(len(series) + 1)]
    for i in range(10000):
        if i % 10 == 0:
            print 'Game: {0}'.format(i)
            probs = measure_performance_vs_random(p1, p2)
            writer.writerow(probs)
            f.flush()
            perf[0].append(i)
            for idx,x in enumerate(probs):
                perf[idx+1].append(x)
        winner = play(p1,p2)
        p1.episode_over(winner)
        #winner = play(r1,p2)
        p2.episode_over(winner)
    f.close()
    for i in range(1,len(perf)):
        plt.plot(perf[0], perf[i], label=series[i-1], color=colors[i-1])
    plt.xlabel('Episodes')
    plt.ylabel('Probability')
    plt.title('RL Agent Performance vs. Random Agent\n({0} loss value, self-play)'.format(p1.lossval))
    #plt.title('P1 Loss={0} vs. P2 Loss={1}'.format(p1.lossval, p2.lossval))
    plt.legend()
    #plt.show()
    #plt.savefig('p1loss{0}vsp2loss{1}.png'.format(p1.lossval, p2.lossval))
    plt.savefig('selfplay_random_{0}loss.png'.format(p1.lossval))
    while True:
        p2.verbose = True
        p1 = Human(1)
        winner = play(p1,p2)
        p1.episode_over(winner)
        p2.episode_over(winner)
'''

'\nif __name__ == "__main__":\n    p1 = Agent(1, lossval = -1)\n    p2 = Agent(2, lossval = -1)\n    r1 = Agent(1, learning = False)\n    r2 = Agent(2, learning = False)\n    r1.epsilon = 1\n    r2.epsilon = 1\n    series = [\'P1-Win\',\'P1-Lose\',\'P1-Draw\',\'P2-Win\',\'P2-Lose\',\'P2-Draw\']\n    #series = [\'P1-Win\', \'P2-Win\', \'Draw\']\n    colors = [\'r\',\'b\',\'g\',\'c\',\'m\',\'b\']\n    markers = [\'+\', \'.\', \'o\', \'*\', \'^\', \'s\']\n    f = open(\'results.csv\', \'wb\')\n    writer = csv.writer(f)    \n    writer.writerow(series)\n    perf = [[] for _ in range(len(series) + 1)]\n    for i in range(10000):\n        if i % 10 == 0:\n            print \'Game: {0}\'.format(i)\n            probs = measure_performance_vs_random(p1, p2)\n            writer.writerow(probs)\n            f.flush()\n            perf[0].append(i)\n            for idx,x in enumerate(probs):\n                perf[idx+1].append(x)\n        winner = play(p1,p2)\n        p1.episode_over(winner)\n   